In [1]:
import pandas as pd
import numpy as np
from keras.models import load_model
import cornac.models.recommender as cmr
import math

Using TensorFlow backend.


In [2]:
def Eat_recommendation():
    #Eat_contents_based

    return;

In [15]:
#Go_contents_based
def Go_recommendation(user_id):
    # 0. preprocess place_matrix
    place_matrix = pd.read_csv('../../Go/data/normalized_item_matrix.csv')
    cbf = place_matrix[['p_id']]
    del place_matrix['total_count']
    place_matrix.set_index('p_id', inplace=True)
    
    # 1. preprocess user_matrix
    user_matrix = pd.read_csv('../../Go/data/normalized_user_matrix.csv')
    user_id  = 'u1s2&'+user_id
    user_index = user_matrix[user_matrix['u_id']==user_id].index[0]
    del user_matrix['u_id']
    del user_matrix['total_count']
    
    # 2. cbf result
    cbf_result = cosine_similarity(place_matrix, user_matrix, user_index)
    cbf['cbf'] = cbf_result
    cbf.set_index('p_id', inplace=True, drop=True)
    print(cbf)
    print('Got Go - contents_based result')
    
    # 3. ncf result
    
    # 4. make_matrix
    go_info =  pd.read_csv('../../Go/data/item_info.csv')
    go_info = pd.DataFrame(go_info, columns=['p_id','p_name','address'])
    go_info.set_index('p_id', inplace=True, drop=True)
    
    # 5. join more
    go_info = go_info.join(cbf, how='left')
    
    # 6. fillNA with 0
    go_info.fillna(0, inplace=True)
    
    
    # 7. 두개 더한값 결과로
    
    # 8. decs 정렬
    
    return go_info;

In [4]:
#Watch_contents_based
def Watch_recommendation(user_id):
    
    return;

In [3]:
def mag(x): return math.sqrt(sum(i**2 for i in x))

In [4]:
def cosine_similarity(p_m, u_m, user_id):
    cos_sim = []
    
    u_vec = u_m.loc[user_id,:]
    u_size = mag(u_vec)

    for i in range(len(p_m)):
        p_vec = p_m.iloc[i,:]
        p_size = mag(p_vec)
        cos_sim.append(np.dot(p_vec, u_vec)/(p_size*u_size))
        
    return cos_sim

In [40]:
def get_preferred_district(u_id):
    user_survey_df = pd.read_csv('../data/survey_result.csv')
    district = user_survey_df[user_survey_df['u_id'] == u_id]['District'].values[0]
    return district.split(', ')

In [51]:
def Search_Recommendable_Places(matrix, u_id):
    dist_ls = get_preferred_district(u_id)
    matrix.sort_values(by=['cbf'], ascending=False, inplace=True)
    matrix.reset_index(inplace=True)
    for i in range(0, len(matrix)):
        try:
            address = matrix['address'][i]
            for j in dist_ls:
                if j in address:
                    return matrix['p_name'][i];
        except:
            pass

In [53]:
def main():
    u_id = input('Enter User ID: ')
    go = Go_recommendation(u_id)
    print(Search_Recommendable_Places(go, u_id))

In [54]:
main()

Enter User ID: 쥬


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


           cbf
p_id          
f239  0.274559
b209       NaN
e810  0.000000
a211  0.341404
b44   0.131585
...        ...
1482  0.218520
1484  0.274702
1485       NaN
1500  0.249071
1510  0.204560

[1206 rows x 1 columns]
Got Go - contents_based result
에덴도자기


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
